In [1]:
import boto3
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from decimal import Decimal
import json


In [2]:
# Initialize DynamoDB client
dynamodb = boto3.resource('dynamodb')
table_name = "AAPL_StockData"
table = dynamodb.Table(table_name)

In [3]:
# Fetch data from DynamoDB
def fetch_data():
    response = table.scan()  # Scan retrieves all records
    data = response['Items']
    
    # Convert to DataFrame
    df = pd.DataFrame(data)
    
    # Convert numerical fields from Decimal to float
    numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Return', 'Volatility', 'SMA_50', 'SMA_200', 'EMA_50', 'EMA_200', 'RSI_14', 'Sharpe_Ratio', 'Sentiment']
    for col in numeric_columns:
        df[col] = df[col].astype(float)
    
    # Sort by date
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values(by='Date', inplace=True)

    return df


In [4]:
# Preprocess Data
def preprocess_data(df):
    features = ['Open', 'High', 'Low', 'Close', 'Volume', 'Return', 'Volatility', 'SMA_50', 'SMA_200', 'EMA_50', 'EMA_200', 'RSI_14', 'Sharpe_Ratio', 'Sentiment']
    
    # Normalize data
    scaler = MinMaxScaler(feature_range=(0,1))
    df_scaled = scaler.fit_transform(df[features])
    
    # Create sequences for LSTM
    sequence_length = 60  # Use last 60 days to predict risk
    X, y = [], []
    
    for i in range(sequence_length, len(df_scaled)):
        X.append(df_scaled[i-sequence_length:i])  # Past 60 days as input
        y.append(df_scaled[i, 6])  # Predicting 'Volatility' (index 6)

    X, y = np.array(X), np.array(y)
    return X, y, scaler


In [5]:
# Build LSTM Model
def build_lstm_model(input_shape):
    model = Sequential([
        LSTM(units=50, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(units=50, return_sequences=False),
        Dropout(0.2),
        Dense(units=25),
        Dense(units=1)  # Predicting risk (volatility)
    ])
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [6]:
def train_model():
    df = fetch_data()
    X, y, scaler = preprocess_data(df)

    # Split into training and testing sets
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    model = build_lstm_model((X_train.shape[1], X_train.shape[2]))
    
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))
    
    # Save the model
    model.save("lstm_stock_risk_model.h5")

    return model, scaler

In [7]:
if __name__ == "__main__":
    model, scaler = train_model()
    print("LSTM Risk Assessment Model Trained Successfully!")

KeyError: 'Sentiment'